In [ ]:
from util import *
import baseline
import numpy as np

from sklearn.preprocessing import StandardScaler
class OptionalStandardScaler(StandardScaler): # class taken from transfer learning paper
    def __init__(self, on=False):
        super(OptionalStandardScaler, self).__init__(with_mean=True, with_std=True)

In [ ]:
import cPickle as pickle
audio = pickle.load( open( "train_test_sets/train-1_test-1/audio.p", "rb" ) )
test_arousal_filenames = pickle.load( open( "train_test_sets/train-1_test-1/test_arousal_filenames.p", "rb" ) )
test_arousal_audio, test_arousal_labels, dumby_doo_doo = util.split_set(test_arousal_filenames, 0.5, audio)

In [ ]:
feat = util.calcConvnetFeatures([test_arousal_audio[0]])
# concat_test_features = [np.concatenate(np.array(l)) for l in feat]
# pickle.dump(concat_test_features, open("train_test_sets/train-1_test-1/convnetTestConcatFeatures.p", 'wb'))

In [ ]:
import cPickle as pickle

In [ ]:
features = pickle.load( open( "train_test_sets/train-1_test-1/convnetFeatures.p", "rb" ) )
pickle.dump(concat_features, open("train_test_sets/train-1_test-1/convnetConcatFeatures.p", 'wb'))
print np.array(features).shape
print np.array(concat_features).shape


In [ ]:
b = baseline.Model()

In [ ]:
import matplotlib.pyplot as plt
import scipy
predictions = b.clf.predict(b.test_arousal_mfcc)
lw = 2
plt.plot(np.array(range(len(predictions))), predictions, color='cornflowerblue', lw=lw, label='predictions')
plt.plot(np.array(range(len(b.test_arousal_labels))), b.test_arousal_labels, color='darkorange', lw=lw, label='ground truth')
plt.xlabel('data')
plt.ylabel('arousal')
plt.title('Support Vector Regression for Arousal with MFCCs from 0.5 second audio')
plt.legend()
plt.show()
print 'pearson correlation coefficient', (np.corrcoef(predictions, b.test_arousal_labels)[0, 1])

In [ ]:
from util import *
train_valence_audio, dumby_doo_doo, train_valence_labels = split_set(b.train_valence_filenames, 0.5, b.audio)
test_valence_audio, dumby_doo_doo, test_valence_labels = split_set(b.test_valence_filenames, 0.5, b.audio)
valenceCLF = pickle.load( open( "baseline_test_1_valence.sav", "rb" ) )

In [ ]:
predictions = valenceCLF.predict(calcMFCCs(test_valence_audio))
lw = 2
plt.plot(np.array(range(len(predictions))), predictions, color='cornflowerblue', lw=lw, label='predictions')
plt.plot(np.array(range(len(b.test_valence_labels))), b.test_valence_labels, color='darkorange', lw=lw, label='ground truth')
plt.xlabel('data')
plt.ylabel('valence')
plt.title('Support Vector Regression for Valence with MFCCs from 0.5 second audio')
plt.legend()
plt.show()
print 'pearson correlation coefficient', (np.corrcoef(predictions, b.test_valence_labels)[0, 1])

In [ ]:
# This JUST to generate the baseline models
for i in [-1, 0, 1, 2, 3, 4, 5]:
    train_valence_audio, dumby_doo_doo, train_valence_labels = split_set(b.train_valence_filenames, 2**i, b.audio)
    test_valence_audio, dumby_doo_doo, test_valence_labels = split_set(b.test_valence_filenames, 2**i, b.audio)
    train_arousal_audio, train_arousal_labels, dumby_doo_doo = split_set(b.train_arousal_filenames, 2**i, b.audio)
    test_arousal_audio, test_arousal_labels, dumby_doo_doo = split_set(b.test_arousal_filenames, 2**i, b.audio)
    length = 2**i
    len_str = str(length) if length != 0.5 else 'half'
    
    arousal_filename = 'MODEL_arousal_mfcc_'+len_str+'_seconds.sav'
    valence_filename = 'MODEL_valence_mfcc_'+len_str+'_seconds.sav'
    b.train(train_arousal_labels, calcMFCCs(train_arousal_audio), arousal_filename)
    b.train(train_valence_labels, calcMFCCs(train_valence_audio), valence_filename)

In [ ]:
# This generates the convnet features - takes very long
for i in [-1, 0, 1, 2, 3, 4, 5]:
    length = 2**i
    len_str = str(length) if length != 0.5 else 'half'
    train_valence_audio, dumby_doo_doo, train_valence_labels = split_set(b.train_valence_filenames, 2**i, b.audio)
    test_valence_audio, dumby_doo_doo, test_valence_labels = split_set(b.test_valence_filenames, 2**i, b.audio)
    train_arousal_audio, train_arousal_labels, dumby_doo_doo = split_set(b.train_arousal_filenames, 2**i, b.audio)
    test_arousal_audio, test_arousal_labels, dumby_doo_doo = split_set(b.test_arousal_filenames, 2**i, b.audio)
    
    train_arousal_convnet_features = calcConvnetFeatures(train_arousal_audio)
    train_arousal_filename = 'FEATURES_TRAIN_arousal_convnet_'+len_str+'_seconds.sav'
    pickle.dump(train_arousal_convnet_features, open(train_arousal_filename, 'wb'))
    
    train_valence_convnet_features = calcConvnetFeatures(train_valence_audio)
    train_valence_filename = 'FEATURES_TRAIN_valence_convnet_'+len_str+'_seconds.sav'
    pickle.dump(train_valence_convnet_features, open(train_valence_filename, 'wb'))
    
    test_arousal_convnet_features = calcConvnetFeatures(test_arousal_audio)
    test_arousal_filename = 'FEATURES_TEST_arousal_convnet_'+len_str+'_seconds.sav'
    pickle.dump(test_arousal_convnet_features, open(test_arousal_filename, 'wb'))
    
    test_valence_convnet_features = calcConvnetFeatures(test_valence_audio)
    test_valence_filename = 'FEATURES_TEST_valence_convnet_'+len_str+'_seconds.sav'
    pickle.dump(test_valence_convnet_features, open(test_valence_filename, 'wb'))

In [ ]:
# This generates the convnet features for te validation set - takes long
for i in [1, 2, 3, 4, 5]:
    length = 2**i
    len_str = str(length) if length != 0.5 else 'half'
    train_valence_audio, dumby_doo_doo, train_valence_labels = split_set(b.validate_train_valence_filenames, 2**i, b.validate_audio)
    test_valence_audio, dumby_doo_doo, test_valence_labels = split_set(b.validate_test_valence_filenames, 2**i, b.validate_audio)
    train_arousal_audio, train_arousal_labels, dumby_doo_doo = split_set(b.validate_train_arousal_filenames, 2**i, b.validate_audio)
    test_arousal_audio, test_arousal_labels, dumby_doo_doo = split_set(b.validate_test_arousal_filenames, 2**i, b.validate_audio)
    
    train_arousal_convnet_features = calcConvnetFeatures(train_arousal_audio)
    train_arousal_filename = 'FEATURES_TRAIN_arousal_convnet_'+len_str+'_seconds.sav'
    pickle.dump(train_arousal_convnet_features, open(train_arousal_filename, 'wb'))
    
    train_valence_convnet_features = calcConvnetFeatures(train_valence_audio)
    train_valence_filename = 'FEATURES_TRAIN_valence_convnet_'+len_str+'_seconds.sav'
    pickle.dump(train_valence_convnet_features, open(train_valence_filename, 'wb'))
    
    test_arousal_convnet_features = calcConvnetFeatures(test_arousal_audio)
    test_arousal_filename = 'FEATURES_TEST_arousal_convnet_'+len_str+'_seconds.sav'
    pickle.dump(test_arousal_convnet_features, open(test_arousal_filename, 'wb'))
    
    test_valence_convnet_features = calcConvnetFeatures(test_valence_audio)
    test_valence_filename = 'FEATURES_TEST_valence_convnet_'+len_str+'_seconds.sav'
    pickle.dump(test_valence_convnet_features, open(test_valence_filename, 'wb'))

In [ ]:
# This JUST to generate the convnet models
for i in [-1, 0, 1, 2, 3, 4, 5]:
    train_valence_audio, dumby_doo_doo, train_valence_labels = split_set(b.train_valence_filenames, 2**i, b.audio)
    train_arousal_audio, train_arousal_labels, dumby_doo_doo = split_set(b.train_arousal_filenames, 2**i, b.audio)
    
    length = 2**i
    len_str = str(length) if length != 0.5 else 'half'
    
    arousal_filename = 'MODEL_arousal_convnet_'+len_str+'_seconds.sav'
    valence_filename = 'MODEL_valence_convnet_'+len_str+'_seconds.sav'
    
    train_arousal_convnet_features = pickle.load( open( "convnet_features/FEATURES_TRAIN_arousal_convnet_"+len_str+"_seconds.sav", "rb" ) )
    train_valence_convnet_features = pickle.load( open( "convnet_features/FEATURES_TRAIN_valence_convnet_"+len_str+"_seconds.sav", "rb" ) )
    
    b.train(train_arousal_labels, train_arousal_convnet_features, arousal_filename)
    b.train(train_valence_labels, train_valence_convnet_features, valence_filename)

In [ ]:
# This JUST to generate the convnet models for validation set - takes long
for i in [1, 2, 3, 4]:
    train_valence_audio, dumby_doo_doo, train_valence_labels = split_set(b.validate_train_valence_filenames, 2**i, b.validate_audio)
    train_arousal_audio, train_arousal_labels, dumby_doo_doo = split_set(b.validate_train_arousal_filenames, 2**i, b.validate_audio)
    
    length = 2**i
    len_str = str(length) if length != 0.5 else 'half'
    
    arousal_filename = 'MODEL_arousal_convnet_'+len_str+'_seconds.sav'
    valence_filename = 'MODEL_valence_convnet_'+len_str+'_seconds.sav'
    
    train_arousal_convnet_features = pickle.load( open( "validate_convnet_features/FEATURES_TRAIN_arousal_convnet_"+len_str+"_seconds.sav", "rb" ) )
    train_valence_convnet_features = pickle.load( open( "validate_convnet_features/FEATURES_TRAIN_valence_convnet_"+len_str+"_seconds.sav", "rb" ) )
    
    b.train(train_arousal_labels, train_arousal_convnet_features, arousal_filename)
    b.train(train_valence_labels, train_valence_convnet_features, valence_filename)

In [ ]:
def plot_results(length, label_type, modelType='mfcc'):
    len_str = str(length) if length != 0.5 else 'half'
    clf = None
    test_audio = None
    test_labels = None
    if label_type == 'valence':
        test_audio, dumby_doo_doo, test_labels = split_set(b.validate_test_valence_filenames, length, b.validate_audio)
        clf = pickle.load( open( modelType+"_models/MODEL_valence_"+modelType+"_"+len_str+"_seconds.sav", "rb" ) )
        train_filenames = b.train_valence_filenames
    else:
        test_audio, test_labels, dumby_doo_doo = split_set(b.validate_test_arousal_filenames, length, b.validate_audio)
        clf = pickle.load( open( modelType+"_models/MODEL_arousal_"+modelType+"_"+len_str+"_seconds.sav", "rb" ) )
        train_filenames = b.train_arousal_filenames
    if modelType == 'mfcc':
        feats = calcMFCCs(test_audio)
        train_audio, dumby_doo_doo, train_labels = split_set(train_filenames, length, b.audio)
        clf = clf.best_estimator_.fit(calcMFCCs(train_audio), train_labels)
    else:
        feats = pickle.load( open( "validate_convnet_features/FEATURES_TEST_"+label_type+"_convnet_"+len_str+"_seconds.sav", "rb" ) )
        train_audio, dumby_doo_doo, train_labels = split_set(train_filenames, length, b.audio)
        train_convnet_features = pickle.load( open( "convnet_features/FEATURES_TRAIN_"+label_type+"_convnet_"+len_str+"_seconds.sav", "rb" ) )
        clf = clf.best_estimator_.fit(train_convnet_features, train_labels)
    predictions = clf.predict(feats)
    lw = 2
    plt.plot(np.array(range(len(predictions))), predictions, color='cornflowerblue', lw=lw, label='predictions')
    plt.plot(np.array(range(len(test_labels))), test_labels, color='darkorange', lw=lw, label='ground truth')
    plt.xlabel('data')
    plt.ylabel(label_type)
    plt.title('Support Vector Regression for '+label_type+' with '+modelType+'\'s from '+len_str+' second audio')
    plt.legend()
    plt.show()
    print 'r squared:', (np.corrcoef(predictions, test_labels)[0, 1] ** 2)
    print 'r:', (np.corrcoef(predictions, test_labels)[0, 1])
    print 'mse:', np.mean(((predictions - test_labels) ** 2))

In [ ]:
plot_results(.5, 'valence', 'mfcc')

In [ ]:
plot_results(.5, 'arousal', 'mfcc')

In [ ]:
import matplotlib.pyplot as plt
clf = pickle.load( open( "convnet_test_1_arousal.sav", "rb" ) )
predictions = clf.predict(concat_test_features)
lw = 2
plt.plot(np.array(range(len(predictions))), predictions, color='cornflowerblue', lw=lw)
plt.plot(np.array(range(len(test_arousal_labels))), test_arousal_labels, color='darkorange', lw=lw)
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression with ConvNew Features from 0.5 second audio')
plt.legend()
plt.show()

In [ ]:
#file_names, audio, convnet_feature_prefix, convnet_model_prefix, mfcc_model_prefix
data_info = {
    "valence_1_train":
    [b.train_valence_filenames, 
     b.audio,
     "convnet_features/FEATURES_TRAIN_valence_convnet_",
     "convnet_models/MODEL_valence_convnet_",
     "mfcc_models/MODEL_valence_mfcc_"
    ],
    "valence_1_test":
    [b.test_valence_filenames, 
     b.audio,
     "convnet_features/FEATURES_TEST_valence_convnet_",
     "convnet_models/MODEL_valence_convnet_",
     "mfcc_models/MODEL_valence_mfcc_"
    ],
    "valence_2_train":
    [b.validate_train_valence_filenames, 
     b.validate_audio,
     "validate_convnet_features/FEATURES_TRAIN_valence_convnet_",
     "convnet_models/MODEL_valence_convnet_",
     "mfcc_models/MODEL_valence_mfcc_"
    ],
    "valence_2_test":
    [b.validate_test_valence_filenames, 
     b.validate_audio,
     "validate_convnet_features/FEATURES_TEST_valence_convnet_",
     "convnet_models/MODEL_valence_convnet_",
     "mfcc_models/MODEL_valence_mfcc_"
    ],
    "arousal_1_train":
    [b.train_arousal_filenames, 
     b.audio,
     "convnet_features/FEATURES_TRAIN_arousal_convnet_",
     "convnet_models/MODEL_arousal_convnet_",
     "mfcc_models/MODEL_arousal_mfcc_"
    ],
    "arousal_1_test":
    [b.test_arousal_filenames, 
     b.audio,
     "convnet_features/FEATURES_TEST_arousal_convnet_",
     "convnet_models/MODEL_arousal_convnet_",
     "mfcc_models/MODEL_arousal_mfcc_"
    ],
    "arousal_2_train":
    [b.validate_train_arousal_filenames, 
     b.validate_audio,
     "validate_convnet_features/FEATURES_TRAIN_arousal_convnet_",
     "convnet_models/MODEL_arousal_convnet_",
     "mfcc_models/MODEL_arousal_mfcc_"
    ],
    "arousal_2_test":
    [b.validate_test_arousal_filenames, 
     b.validate_audio,
     "validate_convnet_features/FEATURES_TEST_arousal_convnet_",
     "convnet_models/MODEL_arousal_convnet_",
     "mfcc_models/MODEL_arousal_mfcc_"
    ],
}

In [ ]:
#returns feat_names, candidate_svms, train_feature_vectors, test_feature_vectors,...
#validate_feature_vectors, train_label_vectors, test_label_vectors, validate_label_vectors
#can be this/other_2/1_train/test
def prepare_multiple(
    train_config=['this_1_train',], 
    test_config=['this_1_test'],
    validate_config=['this_2_test'],
    label_type="arousal"):
    
    other_type = "valence" if label_type == "arousal" else "arousal"
    train_set_config = [spec.replace("this", label_type).replace("other", other_type) for spec in train_config]
    test_set_config = [spec.replace("this", label_type).replace("other", other_type) for spec in test_config]
    validate_set_config = [spec.replace("this", label_type).replace("other", other_type) for spec in validate_config]
    
    feat_names = []
    candidate_svms = []
    train_feature_vectors = []
    test_feature_vectors = []
    validate_feature_vectors = []
    train_label_vectors = []
    test_label_vectors = []
    validate_label_vectors = []
    
    for i in [-1, 0, 1, 2, 3, 4]:
        length = 2**i
        len_str = str(length) if length != 0.5 else 'half'
        
        print("processing " + str(i) + ", will end at 4.")
        
        #prepare train set
        train_mfcc_features = []
        train_convnet_features = []
        train_labels = []
        print "\t (1/3) preparing train set... "
        for train_model_spec in train_set_config:
            [file_names, raw_audio, convnet_feature_prefix, convnet_model_prefix, mfcc_model_prefix] = data_info[train_model_spec]
            split_audio, arousal_labels, valence_labels = split_set(file_names, 2**i, raw_audio)
            labels = arousal_labels if label_type == "arousal" else valence_labels
            if length >= 2:
                convnet_features = pickle.load( open( convnet_feature_prefix+len_str+"_seconds.sav", "rb" ) )
                train_convnet_features = train_convnet_features + convnet_features
            mfcc_features = calcMFCCs(split_audio)
            train_mfcc_features = train_mfcc_features + mfcc_features
            train_labels = train_labels + labels
        #prepare test set
        test_mfcc_features = []
        test_convnet_features = []
        test_labels = []
        print "\t (2/3) preparing test set... "
        for test_model_spec in test_set_config:
            [file_names, raw_audio, convnet_feature_prefix, convnet_model_prefix, mfcc_model_prefix] = data_info[test_model_spec]
            split_audio, arousal_labels, valence_labels = split_set(file_names, 2**i, raw_audio)
            labels = arousal_labels if label_type == "arousal" else valence_labels
            if length >= 2:
                convnet_features = pickle.load( open( convnet_feature_prefix+len_str+"_seconds.sav", "rb" ) )
                test_convnet_features = test_convnet_features + convnet_features
            mfcc_features = calcMFCCs(split_audio)
            test_mfcc_features = test_mfcc_features + mfcc_features
            test_labels = test_labels + labels
        #prepare validate set
        validate_mfcc_features = []
        validate_convnet_features = []
        validate_labels = []
        print "\t (3/3) preparing validate set... "
        for validate_model_spec in validate_set_config:
            [file_names, raw_audio, convnet_feature_prefix, convnet_model_prefix, mfcc_model_prefix] = data_info[validate_model_spec]
            split_audio, arousal_labels, valence_labels = split_set(file_names, 2**i, raw_audio)
            labels = arousal_labels if label_type == "arousal" else valence_labels
            if length >= 2:
                convnet_features = pickle.load( open( convnet_feature_prefix+len_str+"_seconds.sav", "rb" ) )
                validate_convnet_features = validate_convnet_features + convnet_features
            mfcc_features = calcMFCCs(split_audio)
            validate_mfcc_features = validate_mfcc_features + mfcc_features
            validate_labels = validate_labels + labels
            
        if length >= 2:
            convnet_clf = pickle.load( open( "convnet_models/MODEL_"+label_type+"_convnet_"+len_str+"_seconds.sav", "rb" ) )
            candidate_svms.append(convnet_clf)
            feat_names.append("convnet_"+ len_str)
            train_convnet_features, test_convnet_features, validate_convnet_features = norm_with_validate(
                train_convnet_features, test_convnet_features, validate_convnet_features)
            train_feature_vectors.append(train_convnet_features)
            test_feature_vectors.append(test_convnet_features)
            validate_feature_vectors.append(validate_convnet_features)
            train_label_vectors.append(train_labels)
            test_label_vectors.append(test_labels)
            validate_label_vectors.append(validate_labels)
            
        mfcc_clf = pickle.load( open( "mfcc_models/MODEL_"+label_type+"_mfcc_"+len_str+"_seconds.sav", "rb" ) )
        candidate_svms.append(mfcc_clf)
        feat_names.append("mfcc_"+ len_str)
        train_mfcc_features, test_mfcc_features, validate_mfcc_features = norm_with_validate(
                train_mfcc_features, test_mfcc_features, validate_mfcc_features)
        train_feature_vectors.append(train_mfcc_features)
        test_feature_vectors.append(test_mfcc_features)
        validate_feature_vectors.append(validate_mfcc_features)
        train_label_vectors.append(train_labels)
        test_label_vectors.append(test_labels)
        validate_label_vectors.append(validate_labels)
    print "DONE"
    return feat_names, candidate_svms, train_feature_vectors, test_feature_vectors, validate_feature_vectors, train_label_vectors, test_label_vectors, validate_label_vectors



In [ ]:
feat_names, candidate_svms, train_feature_vectors, test_feature_vectors, validate_feature_vectors, train_label_vectors, test_label_vectors, validate_label_vectors = prepare_multiple()




In [ ]:
selected_feature_indexes, best_performance, best_svm = forward_select(candidate_svms, 
                                                                      train_feature_vectors, 
                                                                      train_label_vectors, 
                                                                      test_feature_vectors, 
                                                                      test_label_vectors, 
                                                                      "mse")

In [ ]:
[feat_names[idx] for idx in selected_feature_indexes]

In [ ]:
def plot_from_forward_selection(feat_indexes, model, label_type="arousal"):
    feature_vectors_validate = [validate_feature_vectors[i] for i in feat_indexes]
    label_vectors_validate = [validate_label_vectors[i] for i in feat_indexes]
    feature_vectors_validate, label_vectors_validate = trim(feature_vectors_validate, label_vectors_validate)
    feature_vectors_train = [train_feature_vectors[i] for i in feat_indexes]
    label_vectors_train = [train_label_vectors[i] for i in feat_indexes]
    feature_vectors_train, label_vectors_train = trim(feature_vectors_train, label_vectors_train)
    model.fit(feature_vectors_train, label_vectors_train)
    predictions = model.predict(feature_vectors_validate)
    lw = 2
    plt.plot(np.array(range(len(predictions))), predictions, color='cornflowerblue', lw=lw, label='predictions')
    plt.plot(np.array(range(len(label_vectors_validate))), label_vectors_validate, color='darkorange', lw=lw, label='ground truth')
    plt.xlabel('data')
    plt.ylabel(label_type)
    plt.title('Support Vector Regression for '+label_type + ' with forward selection')
    plt.legend()
    plt.show()
    print 'r squared:', (np.corrcoef(predictions, label_vectors_validate)[0, 1] ** 2)
    print 'r:', (np.corrcoef(predictions, label_vectors_validate)[0, 1])
    print 'mse:', np.mean(((predictions - label_vectors_validate) ** 2))

In [ ]:
plot_from_forward_selection(selected_feature_indexes, best_svm)

In [ ]:
def trim_without_flatten(feature_list, label_list):
    feature_vector = []
    label_vector = []

    shortest_length = min([len(l) for l in label_list])
    for l in label_list:
        if len(l) == shortest_length:
            label_vector = l

    # For every feature in the shortest list, concatenate the
    # appropriate feature from each list
    for feature_index in range(len(feature_list)):
        trimmed_list = []
        curr_list = feature_list[feature_index]
        
        for list_index in range(shortest_length):
            new_idx = ((list_index + 1) * int(len(curr_list) / shortest_length)) - 1
            new_item = curr_list[new_idx]
            trimmed_list.append(new_item)

        feature_vector.append(trimmed_list)

    return feature_vector, label_vector

In [ ]:
#expand
def expand_with_delay(feature_list, label_list, names):
    feature_vector = []
    label_vector = []

    label_vector = label_list[names.index('mfcc_half')]

    for name in names:
        current_feat = feature_list[names.index(name)]
        split_name = name.split("_")
        feature_type = split_name[0]
        length = 0.5 if split_name[1] == "half" else float(split_name[1])
        num_to_remove = int((16.0 / length) - 1)
        current_feat = current_feat[num_to_remove:]
        duplicate_num = int(length / 0.5)
        duplicated = []
        for single_feat in current_feat:
            for i in range(duplicate_num):
                duplicated.append(single_feat)
        duplicated = duplicated[4:] if feature_type == "mfcc" else duplicated[:-4]
        feature_vector.append(duplicated)

    label_vector = label_vector[32:] #get rid of beggining before longest prediction
    label_vector = label_vector[4:] #get rid of 4 samples to simulate 2 second delay later
    feature_vector = [feat[:len(label_vector)] for feat in feature_vector]
    return feature_vector, label_vector

In [ ]:
# prepare data for the multi-layer svr selection process
trimmed_train_feat, trimmed_train_label = trim_without_flatten(train_feature_vectors, train_label_vectors)
trimmed_test_feat, trimmed_test_label = trim_without_flatten(test_feature_vectors, test_label_vectors)
trimmed_validate_feat, trimmed_validate_label = trim_without_flatten(validate_feature_vectors, validate_label_vectors)

In [ ]:
# prepare data for the multi-layer svr selection process with delay to simulate real time operation
trimmed_train_feat, trimmed_train_label = expand_with_delay(train_feature_vectors, train_label_vectors, feat_names)
trimmed_test_feat, trimmed_test_label = expand_with_delay(test_feature_vectors, test_label_vectors, feat_names)
trimmed_validate_feat, trimmed_validate_label = expand_with_delay(validate_feature_vectors, validate_label_vectors, feat_names)

In [ ]:
all_train_predictions = []
all_test_predictions = []
all_validate_predictions = []
for i in range(len(trimmed_train_feat)):
    model = candidate_svms[i]
    model.best_estimator_.fit(trimmed_train_feat[i], trimmed_train_label)
    train_predictions = model.predict(trimmed_train_feat[i])
    test_predictions = model.predict(trimmed_test_feat[i])
    validate_predictions = model.predict(trimmed_validate_feat[i])
    all_train_predictions.append(train_predictions)
    all_test_predictions.append(test_predictions)
    all_validate_predictions.append(validate_predictions)
best_hyper_model, input_models = backwardSelect(feat_names, all_train_predictions, trimmed_train_label, all_test_predictions, trimmed_test_label, "r2")

In [ ]:
input_models

In [ ]:
input_indexes = [feat_names.index(item) for item in input_models]
pred_vec = getPredictionVector(all_validate_predictions, input_indexes)
predictions = best_hyper_model.predict(pred_vec)
lw = 2
plt.plot(np.array(range(len(predictions))), predictions, color='cornflowerblue', lw=lw, label='predictions')
plt.plot(np.array(range(len(trimmed_validate_label))), trimmed_validate_label, color='darkorange', lw=lw, label='ground truth')
plt.xlabel('data')
plt.ylabel('arousal')
plt.title('Support Vector Regression for valence with backward selection')
plt.legend()
plt.show()
print 'r squared:', (np.corrcoef(predictions, trimmed_validate_label)[0, 1] ** 2)
print 'r:', (np.corrcoef(predictions, trimmed_validate_label)[0, 1])
print 'mse:', np.mean(((predictions - trimmed_validate_label) ** 2))